<a href="https://colab.research.google.com/github/ragalaa/Machine-Learning-Basics/blob/master/SQLITE_MD5_file_changesscript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import sqlite3
import hashlib

In [0]:
def getbasefile():
    """ Name of SQLite DB file
        Return name of SQLdatabase file which has the same
        name as python script
     """
    return os.path.splitext(os.path.basename(__file__))[0]

In [0]:
def connectdb():
    """ Connect to SQLite DB """
    try:
        dbfile = getbasefile() + '.db'
        conn = sqlite3.connect(dbfile, timeout=2)
    except  err:
        print (err)
        conn = None
    return conn        

In [0]:
## Write to data base
def corecursor(conn,query):
    """ Opens SQLDB cursor """
    result = False
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fethall()
        numrows = len(list(rows))
        if numrows >0:
            result = True
    except err:
        print (err)
        cursor.close()
    finally:
        cursor.close()
    return result                

In [0]:
def tableexists(table):
    """ Checks if a SQlite DB table exists"""
    result = False
    try:
        conn = connectdb()
        if not conn is None:
            core = "SELECT name FROM sqlite_master WHERE type= 'table' AND name= ' query = core + table +'"
            result = corecursor(conn, query)
            conn.close()
    except err:
        print (err)
        conn.close()
    return result            

In [0]:
def createhashtableidx():
    """ create a SQlite DB Tbale index"""
    table = 'files'
    query = 'CREATE INDEX idxfile on files (files, md5)'
    try:
        conn = connectdb()
        if not conn is None:
            if not tableexists(table):
                try:
                    cursor = conn.cursor()
                    cursor.execute(query)
                except sqlite3.OperationalError:
                    cursor.close()
                finally:
                    conn.commit()
                    conn.close()
    except sqlite3.OperationalError:
        # print str(err) 
        conn.close()  
    finally:
        conn.close()                        

In [0]:
def runcmd(qry):
    """ Run a specific command on the SQLite DB"""
    try:
        conn = connectdb()
        if not conn is None:
            if tableexists('files'):
                try:
                    cursor = conn.cursor()
                    cursor.execute(qry)
                except sqlite3.OperationalError:
                 cursor.close()    
                finally:
                    conn.commit()
                    conn.close()
    except sqlite3.OperationalError:
        # print str(err)
        conn.close()
    finally:
        conn.close()                    

In [0]:
def updatehashtable(fname,md5):
    """ Update the SQlite File tbale"""
    qry = "UPDATE files SET md5 = '{md5}' WHERE file ='{fname}'".format(fname = fname,md5 =md5)
    runcmd(qry)

In [0]:
def inserthashtable(fname,md5):
    """ Insert the SQlite File tbale"""
    qry = "INSERT INTO files (file,md5) VALUES ('{fname}','{md5}')".format(fname = fname,md5 =md5)
    runcmd(qry)

In [0]:
def sethashtable(fname,md5):
    """ Set's up has table"""
    createhashtable()
    createhashtableidx()
    inserthashtable(fname,md5)

In [0]:
def md5indb(fname):
    """ Check if the md5 hash tag exists in the SQlite DB """
    items = []
    qry = "SELECT md5 FROM files WHERE file = '" + fname + "'"
    try:
        conn = connectdb()
        if not conn is None:
            if tableexists('files'):
                try:
                    cursor = conn.cursor()
                    cursor.execute(qry)
                    for row in cursor:
                        items.append(row[0])
                except sqlite3.OperationalError:
                    #print str(err)
                    cursor.close()
                finally:
                    cursor.close()
    except sqlite3.OperationalError:
        # print str(err)
        conn.close()
    finally:
        conn.close()
    return items                                    


In [0]:
 def haschanged(fname,md5):
     """ checkst if filehas changed"""
     result = False
     oldmd5 = md5indb(fname)
     numits = len(oldmd5)
     if numits > 0:
         if oldmd5[0] != md5:
             result = True
             updatehashtable(fname,md5)
     else:
        setuphashtable(fname,md5) 
     return result           

In [0]:
def getfileext(fname):
 # Get the file name extension """

   return os.path.splittext(fname)[1]

In [0]:
def getmoddate(fname):
    """ Get file modified date """
    try:
        mtime = os.path.getmtime(fname)
    except OSError:
        # print str(emsg)
        mtime = 0
    return mtime        

In [0]:
def md5short(fname):
    """ Get md5 file hash tag"""
    return hashlib.md5(fname + '|'+ str(getmoddate(fname))).hexdigest()

In [0]:
def checkfilechanges(folder,exclude):
    """ Checks for files changes"""
    for subdir,dirs,files in os.walk(folder):
        for fname in files:
            origin = os.path.join(subdir, fname)
            if os.path.isfile(origin):
                fext = getfileext(origin)
                if not fext in exclude:
                    md5 = md5short(origin)
                    if haschanged(origin,md5):
                       print(origin + 'has CHnaged...')
                    else:
                        print(origin +' has not changed')   

In [0]:
checkfilechanges("./SAP Extract", ['.db','.py'])